<a href="https://colab.research.google.com/github/G-Gaddu/Quant-Connect/blob/main/Opening%20Range%20Breakout%20Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import the packages
from AlgorithmImports import *

class OpeningRangeBreakout(QCAlgorithm):

    def initialize(self):
        # Initialise the parameters for the algorithm
        self.set_start_date(2024, 1, 1) # Set the start  date
        self.set_end_date(2025, 1, 1) # Set the end date
        self.set_cash("USD",10000000) # Set the strategy cash

        # Define the parameters to be used in the strategy
        self.max_position = 30 # Maximum positions to be held at any time
        self._potential_investments = self.get_parameter("Size_of_Universe", 2000) # Parameter to determine number of potential stocks we will look at
        self._leverage_factor = 5 # Set leverage applied in  strategy
        self._lookback_period = 20 # Indicator period in days
        self._momentum_window= self.get_parameter("Momentum_Window", 10) # Timeframe over which momentum is determined
        self.equity_risk = 0.05 # Equity risk per position
        self.atr_entry_gap = 0.1 # The ATR percentage of the closing price to trigger a stop order
        self._atr_stop_loss = 0.5 # ATR  stop loss threshold (%)
        self._data_by_symbol = {}

        # The S&P500 is added to act as a stepping asset as well as a benchmark
        self._sp500 = self.add_equity("SPY").symbol

        # Set up a warm up period for the strategy
        self.set_warm_up(timedelta(days=2 * self._lookback_period))

        # Set the portfolio to liquidate 1 minute before the market closes each day
        self.schedule.on(
            self.date_rules.every_day(self._sp500),
            self.time_rules.before_market_close(self._sp500, 1),
            self.liquidate
        )

        # Define our universe of the filtered stocks and ensure that the leverage is applied to it
        self.universe_settings.leverage = self._leverage_factor
        self._universe = self.add_universe(self.filtered_universe)
        self.last_month = None

    def on_securities_changed(self, changed):
        # In the event of any changes in the investment universe
        # Apply  labels to each stock that enters the investment universe
        for security in changed.added_securities:
            self._data_by_symbol[security.symbol] = Data(self, security, self._momentum_window, self._lookback_period)

    def filtered_universe(self, fundamentals):
        # Filter the universe based on price and liquidity
        # Update the investment universe on the first day of the month
        if self.time.month == self.last_month:
            return Universe.UNCHANGED
        self.last_month = self.time.month
        # Return the filtered stocks, those that are more than $10 and not the index of the S&P500, sort by dollar volume to get the most liquid stocks and return the first 2000
        return [x.symbol for x in sorted([x for x in fundamentals if x.price > 10 and x.symbol != self._sp500],
                key=lambda x: x.dollar_volume,
                reverse=True)[:self._potential_investments]
                ]

    def on_order_event(self, orderEvent):
        # Handles order events and updates data accordingly
        # If the order is not filled then stop the process
        if orderEvent.status != OrderStatus.FILLED:
            return
        # If the order event for the symbol is found in the data then trigger the order event for that symbol's data
        if orderEvent.symbol in self._data_by_symbol:
            self._data_by_symbol[orderEvent.symbol].on_order_event(orderEvent)

    def on_data(self, slice):
        # Process incoming market data and identify trades
        # First ensure that the strategy does not occur during the warm up period or during the time momentum is being calculated
        if self.is_warming_up or not (self.time.hour == 9 and self.time.minute == 30 + self._momentum_window):
            return
        # Filter the stocks based on the appropriate criteria and get the data for each stock
        filtered_stocks = sorted([self._data_by_symbol[x] for x in self.active_securities.keys
             if self.active_securities[x].price > 0 and x in self._universe.selected
             and self._data_by_symbol[x].ATR.current.value > self._atr_stop_loss
             and self._data_by_symbol[x].rel_volume > 1],
            key=lambda x: x.rel_volume,
            reverse=True
        )[:self.max_position]

        # Look for trade entries
        for data in filtered_stocks:
            data.scan()

class Data:
    def __init__(self, algorithm: QCAlgorithm, security,  momentum_window, lookback_period):
        # Initialise the parameters for the class to track individual stocks
        self.security = security
        self.algorithm = algorithm
        self.SMA_volume = SimpleMovingAverage(lookback_period)
        self.ATR = algorithm.ATR(security.symbol, lookback_period, resolution=Resolution.DAILY)
        self.rel_volume = 0
        self.stop_loss_threshold = None
        self.opening_point = None
        self.entry_ticket = None
        self.stop_loss_ticket = None
        self.consolidator = algorithm.consolidate(security.symbol, TimeSpan.from_minutes(momentum_window), self.consolidate_data_update)

    def scan(self):
        # Looks for trade opportunities based on the opening range breakout
        if not self.opening_point:
            return
        if self.opening_point.close < self.opening_point.open:
            self.make_trade(self.opening_point.low, self.opening_point.low + self.algorithm.atr_entry_gap * self.ATR.current.value)
        elif self.opening_point.close > self.opening_point.open:
            self.make_trade(self.opening_point.high, self.opening_point.high - self.algorithm.atr_entry_gap * self.ATR.current.value)

    def consolidate_data_update(self, point):
        # Update the volume and ATR based on the consolidated data
        if self.opening_point and self.opening_point.time.date() == point.time.date():
            return

        self.SMA_volume.update(point.end_time, point.volume)
        self.rel_volume = point.volume / self.SMA_volume.current.value if self.SMA_volume.is_ready and self.SMA_volume.current.value > 0 else 0
        self.opening_point = point

    def make_trade(self, entry_price, stop_price):
        # Executes a trade based on the entry and stop loss price
        trade_risk = (self.algorithm.portfolio.total_portfolio_value * self.algorithm.equity_risk) / self.algorithm.max_position
        amount = int(trade_risk / (entry_price - stop_price))

        # Limit quantity to what is allowed by portfolio allocation
        limit = self.algorithm.calculate_order_quantity(self.security.symbol, 1 / self.algorithm.max_position)
        if amount < 0:
            sign = -1
        elif amount > 0:
            sign = 1
        else:
            sign = 0
        amount = int(min(abs(amount), limit) * sign)

        if amount != 0:
            self.stop_loss_price = stop_price
            self.entry_ticket = self.algorithm.stop_market_order(self.security.symbol, amount, entry_price, "Entry")

    def on_order_event(self, orderEvent):
        # Handles order events and manages stop loss events
        if self.entry_ticket and orderEvent.order_id == self.entry_ticket.order_id:
            self.stop_loss_ticket = self.algorithm.stop_market_order(
                self.security.symbol, -self.entry_ticket.quantity, self.stop_loss_price, "Stop Loss"
            )